In [2]:
#libraries
import re
import string
from collections import Counter
import numpy as np

# IDENTIFYING THE MISPELLED WORDS

### Step 01

<u>Defining a function for reading a file:</u>

* The name of the function is read_text where we will pass a file name.
* We will open the file for reading as file.
* The variable lines will read the lines that are there in the textfile.
* The word variable is an empty list.
* Next we will create a for loop where we will go through every line from lines and use regular expression.
* The findall() function scans the string from left to right and finds all the matches of the pattern in the string. So this regular expression will break the strings into words for the sentences (ie) we are interested in words and not sentences. 
* Another important thing that we will do is convert the word to lower cases. It always better to stick to one case (either lower or upper case).
* After doing the above steps we upload the text file we want and find the total words present in the text.

In [2]:
def read_text(filename):
    with open(filename,"r") as file:
        lines = file.readlines()
        words = []
        
        for line in lines:
            words = words+ re.findall(r'\w+', line.lower())
    return words

In [3]:
words = read_text("./sonnets.txt")
print(f"There are {len(words)} total words in the text")

There are 17976 total words in the text


### Step 02

<u>Setting the vocabulary:</u>

* The main aim of setting this is for getting unique words from the text.
* Here we will be using the in-built feature of python (ie) set. 
* Why set? Set is a collection of items where elements are present only once (ie) no repeatation allowed.
* After that we print the number of unique words.
* Our autocorrect system will work with this set of words (ie) vocabulary.
* Next we wish to know how many times each word has occured. It is from this we will be building the word probabilities that will help us to select the suggested candidates to fix the misspelled words.
* For this purpose we will the **Counter()**
* Counter is just like dictionnary but what it does it that you give a list of items, it will form a dictionnary where items are keys and frequencies are the values.

In [4]:
# creating a vocabulary set
vocab = set(words)
print(f"{len(vocab)} unique words exists in the vocabulary")

3086 unique words exists in the vocabulary


In [5]:
# for creating word probabilities we need to know how many times each word appears
# Here we use the Counter which will return a dictionnary where items are keys and frequencies are the values. 
count_words = Counter(words)

# randomly let's put a word and check how many times it appears. Here i took the particular word as world.
print(count_words["world"])

33


# FINDING STRING THAT ARE n EDIT DISTANCE AWAY

### Step 01

<u>Calculating the word probabilities:</u>
* Initially we will find the total number of words in the text. This will be same as the lenght of the words that was done in the first step. 
* For creating the word probabilities we will use the dictionary comprehensions.
* The probability to calculate the word probability is $P(w) = \frac{c(w)}{V}$ this formula indicated by prob_word.
* We can check this too. But while checking keep it in mind that the words that we want to check should be typed in lower case as we have defined in the above steps that it should be in lower case. If this is ignored then we will get an error

In [6]:
total_word_count = float(sum(count_words.values()))
total_word_count

17976.0

In [7]:
prob_word = {word:count_words[word]/total_word_count for word in count_words.keys()}
print(prob_word['world']) # checking
print(prob_word['thou'])

0.001835781041388518
0.013072986203827325


In [8]:
print(prob_word['Thou']) # ERROR

KeyError: 'Thou'

### Step 02

<u>Type of edit operations</u>
* **Splitting the words into 2 or more components:** here we will use a function and it will return a tuple. Here we will be using list comprehension.

* **Delete:** here we will remove a letter from the above split words.
* **Swap:** here we will swap the 2 adjacent letters.
* **replace:** changes one letter to another. We need letters in english so we will use ASCII.
* **insert:** add a letter

In [9]:
def split(word):
    return [(word[:i],word[i:]) for i in range(len(word)+1)]
print(split("world"))

[('', 'world'), ('w', 'orld'), ('wo', 'rld'), ('wor', 'ld'), ('worl', 'd'), ('world', '')]


In [10]:
def delete(word):
    return [l+r[1:] for l,r in split(word) if r]
print(delete("world"))

['orld', 'wrld', 'wold', 'word', 'worl']


In [11]:
def swap(word):
    return [l+r[1]+r[0]+r[2:] for l,r in split(word) if len(r)>1]
print(swap("world"))

['owrld', 'wrold', 'wolrd', 'wordl']


In [12]:
def replace(word):
    letters = string.ascii_lowercase
    return[l+c+r[1:] for l,r in split(word) if r for c in letters]
print(replace("world"))

['aorld', 'borld', 'corld', 'dorld', 'eorld', 'forld', 'gorld', 'horld', 'iorld', 'jorld', 'korld', 'lorld', 'morld', 'norld', 'oorld', 'porld', 'qorld', 'rorld', 'sorld', 'torld', 'uorld', 'vorld', 'world', 'xorld', 'yorld', 'zorld', 'warld', 'wbrld', 'wcrld', 'wdrld', 'werld', 'wfrld', 'wgrld', 'whrld', 'wirld', 'wjrld', 'wkrld', 'wlrld', 'wmrld', 'wnrld', 'world', 'wprld', 'wqrld', 'wrrld', 'wsrld', 'wtrld', 'wurld', 'wvrld', 'wwrld', 'wxrld', 'wyrld', 'wzrld', 'woald', 'wobld', 'wocld', 'wodld', 'woeld', 'wofld', 'wogld', 'wohld', 'woild', 'wojld', 'wokld', 'wolld', 'womld', 'wonld', 'woold', 'wopld', 'woqld', 'world', 'wosld', 'wotld', 'would', 'wovld', 'wowld', 'woxld', 'woyld', 'wozld', 'worad', 'worbd', 'worcd', 'wordd', 'wored', 'worfd', 'worgd', 'worhd', 'worid', 'worjd', 'workd', 'world', 'wormd', 'wornd', 'worod', 'worpd', 'worqd', 'worrd', 'worsd', 'wortd', 'worud', 'worvd', 'worwd', 'worxd', 'woryd', 'worzd', 'worla', 'worlb', 'worlc', 'world', 'worle', 'worlf', 'worlg', 

In [13]:
def insert(word):
    letters = string.ascii_lowercase
    return [l+c+r for l,r in split(word) for c in letters]
print(insert("world"))

['aworld', 'bworld', 'cworld', 'dworld', 'eworld', 'fworld', 'gworld', 'hworld', 'iworld', 'jworld', 'kworld', 'lworld', 'mworld', 'nworld', 'oworld', 'pworld', 'qworld', 'rworld', 'sworld', 'tworld', 'uworld', 'vworld', 'wworld', 'xworld', 'yworld', 'zworld', 'waorld', 'wborld', 'wcorld', 'wdorld', 'weorld', 'wforld', 'wgorld', 'whorld', 'wiorld', 'wjorld', 'wkorld', 'wlorld', 'wmorld', 'wnorld', 'woorld', 'wporld', 'wqorld', 'wrorld', 'wsorld', 'wtorld', 'wuorld', 'wvorld', 'wworld', 'wxorld', 'wyorld', 'wzorld', 'woarld', 'wobrld', 'wocrld', 'wodrld', 'woerld', 'wofrld', 'wogrld', 'wohrld', 'woirld', 'wojrld', 'wokrld', 'wolrld', 'womrld', 'wonrld', 'woorld', 'woprld', 'woqrld', 'worrld', 'wosrld', 'wotrld', 'wourld', 'wovrld', 'wowrld', 'woxrld', 'woyrld', 'wozrld', 'worald', 'worbld', 'worcld', 'wordld', 'woreld', 'worfld', 'worgld', 'worhld', 'worild', 'worjld', 'workld', 'worlld', 'wormld', 'wornld', 'worold', 'worpld', 'worqld', 'worrld', 'worsld', 'wortld', 'woruld', 'worvld',

### Step 03

<u>One edit distance away</u>


In this we will take unique values from the above edit operations(except split). So we will implement sets here.
Repeating this again 

In [14]:
def level_one_edit(word):
    return set(delete(word)+swap(word)+replace(word)+insert(word))
print(level_one_edit("world"))

{'aorld', 'woprld', 'wqorld', 'worldh', 'worlf', 'woqld', 'would', 'worlq', 'wofrld', 'wolld', 'woorld', 'worlqd', 'woyld', 'kworld', 'orld', 'wojrld', 'worud', 'worlwd', 'horld', 'worled', 'worlde', 'worlhd', 'worrld', 'wormld', 'wvorld', 'wortld', 'word', 'wogrld', 'eorld', 'iorld', 'worsd', 'zworld', 'wuorld', 'korld', 'wogld', 'bworld', 'wyorld', 'worli', 'wvrld', 'oworld', 'wkorld', 'wdorld', 'worold', 'wokrld', 'woreld', 'uorld', 'worldr', 'hworld', 'wtrld', 'wcrld', 'worlmd', 'wolrd', 'worlo', 'worgld', 'zorld', 'qworld', 'wordl', 'worljd', 'worvd', 'worle', 'wrld', 'woryld', 'eworld', 'lorld', 'wodld', 'worlxd', 'jworld', 'worlx', 'wold', 'oorld', 'worl', 'worfd', 'wortd', 'worlg', 'wporld', 'woirld', 'woyrld', 'worvld', 'worlbd', 'wohld', 'worlvd', 'worlnd', 'vorld', 'worlpd', 'worldp', 'worldn', 'worhld', 'forld', 'worldo', 'worldt', 'sorld', 'wbrld', 'woxld', 'woqrld', 'worlcd', 'wosrld', 'norld', 'rworld', 'worxd', 'vworld', 'worlrd', 'weorld', 'wgrld', 'worlod', 'tworld', 

In [15]:
def level_two_edit(word):
    return set(l2 for l1 in level_one_edit(word) for l2 in level_one_edit(l1))
print(level_two_edit("world"))

{'wurjld', 'woolkd', 'worldfl', 'worydu', 'cwporld', 'ttorld', 'worrkld', 'wxrlgd', 'wortla', 'wowrljd', 'toqld', 'worade', 'gold', 'wqorldt', 'whzorld', 'worldcd', 'wiosrld', 'vorgd', 'worold', 'worecd', 'wmould', 'wowrfd', 'hrworld', 'worlbi', 'worldtu', 'worlvld', 'wotjld', 'worlgh', 'yworcd', 'wocrla', 'wogmld', 'wvomrld', 'wtrlb', 'woraad', 'worldoy', 'aworled', 'gbrld', 'sorldw', 'yorlv', 'wbrid', 'wogrlcd', 'woplzd', 'woxrjld', 'worlkl', 'fwormld', 'wofdrld', 'worxbld', 'rworlmd', 'wuorwld', 'bworjld', 'mworkld', 'woumd', 'wotmrld', 'uwxrld', 'worgldd', 'aojld', 'wormv', 'wjorlbd', 'jorld', 'wxovld', 'worlkdm', 'worlado', 'wwtorld', 'wovlp', 'worldzl', 'wjorpld', 'wxorald', 'worlgp', 'wroldy', 'wornlmd', 'wqozld', 'wobldi', 'worltdm', 'worlzdi', 'twocld', 'wpohrld', 'wmorli', 'worilp', 'nwyrld', 'worttld', 'dwokrld', 'woipd', 'worpd', 'jorldc', 'worlkcd', 'worlmdn', 'qworild', 'wogrldm', 'wroldg', 'tworldr', 'zworlbd', 'wztld', 'eworlds', 'worbz', 'eborld', 'vorbd', 'wrorlu', 'w

## FINAL

In [16]:
def spell_check(word,vocabulary,word_prob):
    #identifying the misspelled word
    if word in vocabulary:
        print(f"{word} correctly spelled!")
        return
    suggested = level_one_edit(word) or level_two_edit(word) or[word]
    best_guess = [w for w in suggested if w in vocabulary]
    return [(w, word_prob[w]) for w in best_guess]

In [17]:
word = 'hime'# misspelled a word
guesses = spell_check(word,vocab,prob_word)
print(guesses)

[('him', 0.002113929684023142), ('hide', 0.00027814864263462394), ('home', 0.00016688918558077436), ('time', 0.003894080996884735)]


In [18]:
word = 'foad'# misspelled a word
guesses = spell_check(word,vocab,prob_word)
print(guesses)

[('food', 5.562972852692479e-05), ('fond', 0.00011125945705384957), ('fold', 5.562972852692479e-05)]


## USING OOPS 

In [3]:
class auto_correct(object):
    def __init__(self, corpus_file_path):
        with open(corpus_file_path, "r") as file:
            lines = file.readlines()
            words = []
        for line in lines:
            words += re.findall(r'\w+', line.lower())
        self.vocabs = set(words)# as we need unique words
        self.word_counts = Counter(words)
        total_words = float(sum(self.word_counts.values()))
        self.word_probas = {word: self.word_counts[word] / total_words for word in self.vocabs}

    def _level_one_edits(self, word):
        #we need words in english and in lowercase.
        letters = string.ascii_lowercase
        splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletes = [l + r[1:] for l,r in splits if r]
        swaps = [l + r[1] + r[0] + r[2:] for l, r in splits if len(r)>1]
        replaces = [l + c + r[1:] for l, r in splits if r for c in letters]
        inserts = [l + c + r for l, r in splits for c in letters] 
        return set(deletes + swaps + replaces + inserts)

    def _level_two_edits(self, word):
        return set(e2 for e1 in self._level_one_edits(word) for e2 in self._level_one_edits(e1))

    def check(self, word):
        candidates = self._level_one_edits(word) or self._level_two_edits(word) or [word]
        valid_candidates = [w for w in candidates if w in self.vocabs]
        return sorted([(c, self.word_probas[c]) for c in valid_candidates], key=lambda tup: tup[1], reverse=True)

In [4]:
checker = auto_correct("./sonnets.txt")

In [5]:
#testing
checker.check("hime")

[('time', 0.003894080996884735),
 ('him', 0.002113929684023142),
 ('hide', 0.00027814864263462394),
 ('home', 0.00016688918558077436)]

In [59]:
#checker.check("cas")

[('as', 0.006731197151757899),
 ('can', 0.002447708055184691),
 ('was', 0.0016132621272808188),
 ('case', 0.00016688918558077436),
 ('cast', 0.00011125945705384957),
 ('cars', 5.562972852692479e-05),
 ('car', 5.562972852692479e-05)]

In [60]:
#checker.check("mi")

[('my', 0.02186248331108144),
 ('i', 0.019526034712950602),
 ('me', 0.009123275478415665)]

In [62]:
#checker.check("he")

[('the', 0.02403204272363151),
 ('me', 0.009123275478415665),
 ('be', 0.00789942145082332),
 ('her', 0.0028371161548731644),
 ('he', 0.002447708055184691),
 ('she', 0.001835781041388518),
 ('we', 0.0008344459279038719),
 ('hue', 0.00027814864263462394),
 ('ne', 0.00022251891410769915),
 ('ye', 0.00011125945705384957),
 ('re', 5.562972852692479e-05)]

In [65]:
checker.check("boe")

[('be', 0.00789942145082332),
 ('woe', 0.0006675567423230974),
 ('boy', 0.00016688918558077436),
 ('bow', 0.00011125945705384957),
 ('bore', 0.00011125945705384957),
 ('foe', 5.562972852692479e-05)]